# Global Societal Endangerment Index (GSEI)

Development notebook for step 2: Data Selection

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import pycountry

# Data Loading and Preprocessing
Bring datasets into the right format and perform initial transformations to form the base indicators.

Total indicator count to load: 9 + 7 + 8 + 11 + 4 = 39

In [3]:
"""
Country standardisation functions
"""

UNKNOWN_COUNTRY = 'aaa.Unknown'

def all_countries():
    return [c.name for c in pycountry.countries]

def lookup_country(country_name: str, fuzzy_threshold=80):
    try:
        return pycountry.countries.lookup(country_name).name
    except LookupError: pass
    
    countries = all_countries()
    c_match = [c.casefold() for c in countries]
    try:
        if country_name.casefold() in c_match:
            return countries[c_match.index(country_name.casefold())]
    except Exception: pass
    
    # special cases
    if country_name == 'Canary Islands' or country_name == 'SPI':
        print(f"Using special case for '{country_name}' -> 'Spain'")
        return 'Spain'
    elif country_name == 'Turkey':
        print(f"Using special case for '{country_name}' -> 'Türkiye'")
        return 'Türkiye'
    
    # fuzzy match
    try:
        best_match, score = process.extractOne(country_name, countries)
        if score > fuzzy_threshold:
            print(f"Using fuzzy match for '{country_name}' -> '{best_match}'")
            return best_match
        else:
            raise LookupError(f"Country '{country_name}' not found (best fuzzy match too low: {best_match} ({score}))")
    except LookupError as e:
        print(e)
        return UNKNOWN_COUNTRY

def standardise_countries(country_col: pd.Series, fuzzy_threshold=80):
    return country_col.map(lambda c: lookup_country(c, fuzzy_threshold=fuzzy_threshold))

In [4]:
def transpose_world_bank_data(wb_df, year_range):
    """
    Transpose World Bank data from long to wide format.
    """
    # clean up rows
    # drop ANT, CHI, XKX because mapping them to another country would produce duplicate country rows that are very complicated to merge
    codes_to_remove = ['ANT','CHI','XKX', 
    'AFE','AFW','ARB','CSS','CEB','EAR','EAS','EAP','TEA','EMU','ECS','ECA','TEC','EUU','FCS','HPC','HIC','IBD','IBT','IDB','IDX','IDA','LTE','LCN','LAC','TLA','LDC','LMY','LIC','LMC','MEA','MNA','TMN','MIC','NAC','INX','OED','OSS','PSS','PST','PRE','SST','SAS','TSA','SSF','SSA','TSS','UMC','WLD']
    wb_df = wb_df[~wb_df['Country Code'].isin(codes_to_remove)]
    wb_df = wb_df[~wb_df['Country Name'].str.contains("Data from", na=False)]
    wb_df = wb_df[~wb_df['Country Name'].str.contains("Last Updated", na=False)]
    wb_df = wb_df.dropna(how='all', axis=0)  # drop completely empty rows
    wb_df = wb_df.replace('..', np.nan)  # replace missing values with NaN
    # clean up columns
    wb_df['Country Name'] = standardise_countries(wb_df['Country Code'], fuzzy_threshold=100)
    wb_df = wb_df.drop(columns=['Country Code', 'Series Code'])
    wb_df.columns = [col.split(" [")[0] if "[YR" in col else col for col in wb_df.columns]
    # Pivot to wide format
    wb_df = wb_df.pivot(index='Country Name', columns='Series Name', values=[str(year) for year in year_range])
    wb_df.columns = ['_'.join(col).strip() for col in wb_df.columns.values]
    wb_df = wb_df.reset_index()
    # convert to numeric
    num_cols = wb_df.drop('Country Name', axis=1).apply(pd.to_numeric, errors='coerce')
    wb_df = pd.concat([wb_df['Country Name'], num_cols], axis=1)
    return wb_df

def indicator_yearly_availability(processed_wb_df):
    """
    Calculate the availability of data for each indicator and year.
    """
    # Select indicator columns
    indicator_cols = processed_wb_df.columns[1:]
    # Calculate availability (non-null percentage)
    availability = processed_wb_df[indicator_cols].notnull().mean()
    # Reshape to DataFrame
    availability_df = availability.to_frame(name='value').reset_index()
    availability_df.rename(columns={'index': 'indicator'}, inplace=True)
    # Split indicator column
    availability_df[['year', 'indicatorName']] = availability_df['indicator'].str.split('_', n=1, expand=True)
    # Drop original indicator column
    availability_df.drop('indicator', axis=1, inplace=True)
    # Pivot to desired format
    return availability_df.pivot(index='indicatorName', columns='year', values='value').reset_index()

### Environmental Risks


In [5]:
# start with comprehensive country list
env_data = pd.DataFrame(all_countries(), columns=['Country'])

env_data

,Country
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Åland Islands
...,...
244,Samoa
245,Yemen
246,South Africa
247,Zambia


In [6]:
# Climate Change Vulnerability

# Maximum relative temperature change (°C) in the last 10 years, compared to a 1951-1980 baseline
temp_change = pd.read_csv("data/UN_FAO_climate_change_indicators.csv")
# drop row with Country 'world' in temp_change
temp_change = temp_change[temp_change['Country'] != 'World']
# standardise country names
temp_change['Country'] = standardise_countries(temp_change['ISO3'])
# INDICATOR: calculate maximum value between F2012 and F2022
indicator_cols = ['Temp Change max 2012-2022']
temp_change[indicator_cols[0]] = temp_change[['F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022']].max(axis=1)
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(temp_change[['Country'] + indicator_cols], on='Country', how='left')

print(env_data.columns.values)

['Country' 'Temp Change max 2012-2022']


In [7]:
# Exposure to Natural Disasters (earthquakes, floods, hurricanes)

disasters = pd.read_excel("data/EM-DAT_natural_disasters_ALL_2020-2025.xlsx")
# standardise country names
disasters['Country'] = standardise_countries(disasters['ISO'])
# group by country, summing up INDICATORS: affected people + total damage
disasters_grouped = disasters.groupby('Country')[['No. Affected', 'Total Damage, Adjusted (\'000 US$)']].sum().reset_index()
disasters_grouped['Total Damage, Adjusted (\'000 US$)'] = disasters_grouped['Total Damage, Adjusted (\'000 US$)'] * 1000
# rename columns
indicator_cols = ['Disaster Affected Population 2020-2025', 'Disaster Damage US$ 2020-2025']
disasters_grouped.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(disasters_grouped, on='Country', how='left')

print(env_data.columns.values)

Using special case for 'SPI' -> 'Spain'
Using special case for 'SPI' -> 'Spain'
['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025']


In [8]:
# Air and Water Pollution Levels

# INDICATORS:
# Years of lost life due to unsafe water, sanitation, and handwashing
# Years of lost life due to air pollution

air_water = pd.read_csv("data/IHME_GBD_environmental_risk_export_2021.csv")
# standardise country names
air_water['Location'] = standardise_countries(air_water['Location'], fuzzy_threshold=79)
# rename columns
indicator_cols = ['Unsafe water, sanitation (YLL)', 'Air pollution (YLL)']
air_water.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(air_water, on='Country', how='left')

print(env_data.columns.values)

Using fuzzy match for 'Cape Verde' -> 'Cabo Verde'
Using fuzzy match for 'Cote d'Ivoire' -> 'Côte d'Ivoire'
Using fuzzy match for 'Democratic Republic of Congo' -> 'Congo, The Democratic Republic of the'
Using fuzzy match for 'Iran (Islamic Republic of)' -> 'Iran, Islamic Republic of'
Using fuzzy match for 'Libyan Arab Jamahiriya' -> 'Libya'
Using fuzzy match for 'Republic of Congo' -> 'Congo'
Using fuzzy match for 'Republic of Korea' -> 'Korea, Republic of'
Using special case for 'Turkey' -> 'Türkiye'
['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025'
 'Unsafe water, sanitation (YLL)' 'Air pollution (YLL)']


In [9]:
wb_dev2 = pd.read_csv("data/Worldbank_development_2.csv")
wb_dev2 = transpose_world_bank_data(wb_dev2, range(2014, 2024))

wb_dev2

,Country Name,2014_Access to electricity (% of population),"2014_External debt stocks, total (DOD, current US$)",2014_Individuals using the Internet (% of population),"2014_Inflation, consumer prices (annual %)",2014_People using safely managed drinking water services (% of population),2014_Prevalence of moderate or severe food insecurity in the population (%),2014_Prevalence of undernourishment (% of population),2014_Renewable energy consumption (% of total final energy consumption),2014_Renewable internal freshwater resources per capita (cubic meters),...,"2023_Inflation, consumer prices (annual %)",2023_People using safely managed drinking water services (% of population),2023_Prevalence of moderate or severe food insecurity in the population (%),2023_Prevalence of undernourishment (% of population),2023_Renewable energy consumption (% of total final energy consumption),2023_Renewable internal freshwater resources per capita (cubic meters),2023_Secure Internet servers (per 1 million people),2023_Total reserves (% of total external debt),"2023_Unemployment, total (% of total labor force) (modeled ILO estimate)","2023_Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)"
0,Afghanistan,89.5,2.529865e+09,7.0000,4.673996,22.944301,NaN,19.3,19.1,1437.827763,...,NaN,NaN,NaN,NaN,NaN,NaN,4.375854e+01,NaN,13.991,17.291
1,Albania,100.0,8.512452e+09,54.3000,1.625865,70.563261,NaN,4.5,38.6,9310.845162,...,4.759764,NaN,NaN,NaN,NaN,NaN,1.220333e+03,56.806329,10.108,24.817
2,Algeria,99.3,5.521189e+09,29.5000,2.916927,76.618232,NaN,2.7,0.1,286.876447,...,9.322174,NaN,NaN,NaN,NaN,NaN,1.002941e+02,1110.236304,11.701,30.447
3,American Samoa,NaN,NaN,NaN,NaN,89.165530,NaN,NaN,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.629532e+02,NaN,NaN,NaN
4,Andorra,100.0,NaN,86.1000,NaN,90.640000,NaN,NaN,19.4,4280.076488,...,NaN,NaN,NaN,NaN,NaN,NaN,1.461858e+04,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,"Virgin Islands, British",99.2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.982403e+06,NaN,NaN,NaN
211,"Virgin Islands, U.S.",100.0,NaN,50.0700,NaN,97.936004,NaN,NaN,4.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.050030e+02,NaN,12.320,25.824
212,Yemen,66.1,7.723142e+09,22.5500,8.104726,NaN,NaN,32.4,0.8,69.475899,...,NaN,NaN,NaN,NaN,NaN,NaN,6.955939e+00,NaN,17.091,32.395
213,Zambia,27.9,9.675903e+09,6.5000,7.806876,NaN,NaN,36.0,84.6,5045.511838,...,10.884532,NaN,NaN,NaN,NaN,NaN,5.496053e+01,NaN,5.905,9.752


In [10]:
# check availability of data for each indicator over the years
indicator_yearly_availability(wb_dev2)

year,indicatorName,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Access to electricity (% of population),0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.000000
1,"External debt stocks, total (DOD, current US$)",0.544186,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488
2,Individuals using the Internet (% of population),0.934884,0.930233,0.944186,0.953488,0.813953,0.865116,0.893023,0.883721,0.851163,0.274419
3,"Inflation, consumer prices (annual %)",0.874419,0.869767,0.869767,0.846512,0.832558,0.832558,0.804651,0.804651,0.800000,0.758140
4,People using safely managed drinking water ser...,0.637209,0.637209,0.641860,0.641860,0.637209,0.637209,0.637209,0.627907,0.613953,0.000000
5,Prevalence of moderate or severe food insecuri...,0.000000,0.455814,0.460465,0.539535,0.572093,0.627907,0.683721,0.706977,0.702326,0.000000
6,Prevalence of undernourishment (% of population),0.795349,0.795349,0.795349,0.795349,0.795349,0.790698,0.795349,0.795349,0.795349,0.000000
7,Renewable energy consumption (% of total final...,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.330233,0.000000
8,Renewable internal freshwater resources per ca...,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.000000,0.000000
9,Secure Internet servers (per 1 million people),0.976744,0.986047,0.995349,0.995349,0.990698,0.986047,0.995349,1.000000,1.000000,1.000000


#### Year selection for Worldbank Development Indicators (dataset 2)

Based on the availability of data for the Worldbank Development Indicators, one year is selected for each indicator that balances recency and data availability. The missing data will be handled in the imputation step. The selected years are:

- 2022 (99%) <- Access to electricity (% of population)
- 2023 (55%) <- "External debt stocks, total (DOD, current US$)"
- 2022 (85%) <- Individuals using the Internet (% of population)
- 2023 (75%) <- "Inflation, consumer prices (annual %)"
- 2022 (61%) <- People using safely managed drinking water services (% of population)
- 2022 (70%) <- Prevalence of moderate or severe food insecurity in the population (%)
- 2022 (80%) <- Prevalence of undernourishment (% of population)
- 2021 (98%) <- Renewable energy consumption (% of total final energy consumption)
- 2021 (85%) <- Renewable internal freshwater resources per capita (cubic meters)
- 2023 (100%) <- Secure Internet servers (per 1 million people)
- 2021 (44%) <- Total reserves (% of total external debt)
- 2023 (85%) <- "Unemployment, total (% of total labor force) (modeled ILO estimate)"
- 2023 (85%) <- "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)"


In [11]:
# Water Scarcity & Food Security

# INDICATORS:
# Renewable internal freshwater resources per capita (cubic meters)
# People using safely managed drinking water services (% of population)
# Prevalence of moderate or severe food insecurity in the population (%)
# Prevalence of undernourishment (% of population)

water_food = wb_dev2[['Country Name', '2022_People using safely managed drinking water services (% of population)', '2022_Prevalence of moderate or severe food insecurity in the population (%)', '2022_Prevalence of undernourishment (% of population)', '2021_Renewable internal freshwater resources per capita (cubic meters)']]
# rename columns
indicator_cols = ['Safe Drinking Water (%)', 'Food Insecurity (%)', 'Undernourishment (%)', 'Renewable Freshwater per Capita (m3)']
water_food.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(water_food, on='Country', how='left')

print(env_data.columns.values)

['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025'
 'Unsafe water, sanitation (YLL)' 'Air pollution (YLL)'
 'Safe Drinking Water (%)' 'Food Insecurity (%)' 'Undernourishment (%)'
 'Renewable Freshwater per Capita (m3)']


### Political Instability & Governance


In [12]:
pol_data = pd.DataFrame(all_countries(), columns=['Country'])

In [13]:
# Governance Quality

wb_gov = pd.read_csv('data/Worldbank_governance.csv')
wb_gov = transpose_world_bank_data(wb_gov, range(2014, 2024))

indicator_yearly_availability(wb_gov)

year,indicatorName,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Control of Corruption: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
1,Government Effectiveness: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
2,Political Stability and Absence of Violence/Te...,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000,1.000000,1.000000,1.000000,0.995283
3,Regulatory Quality: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
4,Rule of Law: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
5,Voice and Accountability: Estimate,0.957547,0.957547,0.957547,0.957547,0.971698,0.976415,0.976415,0.976415,0.976415,0.962264


In [14]:
# use 2023 since all data is >95% available

# INDICATORS:
# Corruption Control
# Rule of Law
# Political Stability
# Government Effectiveness
# Regulatory Quality
# Voice and Accountability

governance = wb_gov[['Country Name'] + list(wb_gov.columns[wb_gov.columns.str.startswith('2023')])]
# rename columns
indicator_cols = ['Corruption Control', 'Rule of Law', 'Political Stability', 'Government Effectiveness', 'Regulatory Quality', 'Voice and Accountability']
governance.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in pol_data.columns:
    pol_data = pol_data.drop(columns=indicator_cols)
pol_data = pol_data.merge(governance, on='Country', how='left')

print(pol_data.columns.values)

['Country' 'Corruption Control' 'Rule of Law' 'Political Stability'
 'Government Effectiveness' 'Regulatory Quality'
 'Voice and Accountability']


In [15]:
# INDICATOR: Democracy, not Autocracy

regime = pd.read_csv('data/V-Dem-Institute_ERT_democracy.csv')
# select most recent year
regime = regime[regime['year'] == 2023]
# standardise country names
regime['country_name'] = standardise_countries(regime['country_text_id'])
regime = regime[regime['country_name'] != UNKNOWN_COUNTRY]
# select & rename columns
regime = regime[['country_name', 'reg_type']]
indicator_cols = ['Democracy, not Autocracy']
regime.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in pol_data.columns:
    pol_data = pol_data.drop(columns=indicator_cols)
pol_data = pol_data.merge(regime, on='Country', how='left')

print(pol_data.columns.values)

Country 'XKX' not found (best fuzzy match too low: Denmark (36))
Country 'PSG' not found (best fuzzy match too low: Cyprus (44))
Country 'SML' not found (best fuzzy match too low: Åland Islands (60))
Country 'ZZB' not found (best fuzzy match too low: Uzbekistan (60))
['Country' 'Corruption Control' 'Rule of Law' 'Political Stability'
 'Government Effectiveness' 'Regulatory Quality'
 'Voice and Accountability' 'Democracy, not Autocracy']


### Social Vulnerability


In [16]:
soc_data = pd.DataFrame(all_countries(), columns=['Country'])

### Economic Instability & Infrastructure


In [17]:
eco_data = pd.DataFrame(all_countries(), columns=['Country'])

### Global & Regional Threats


In [18]:
threat_data = pd.DataFrame(all_countries(), columns=['Country'])

In [19]:
# env_data
pol_data

,Country,Corruption Control,Rule of Law,Political Stability,Government Effectiveness,Regulatory Quality,Voice and Accountability,"Democracy, not Autocracy"
0,Aruba,0.709075,0.795441,1.432525,0.946606,1.270207,1.019175,NaN
1,Afghanistan,-1.154932,-1.987014,-2.484081,-1.266425,-1.649641,-1.852901,0.0
2,Angola,-0.609920,-1.008910,-0.341505,-0.761388,-1.099066,-0.737252,0.0
3,Anguilla,1.251356,1.201682,1.114221,0.931280,0.683676,NaN,NaN
4,Åland Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
244,Samoa,0.727086,0.492577,1.098188,-0.270535,0.900673,1.019603,NaN
245,Yemen,-1.648933,-2.275420,-2.562939,-1.843051,-1.838976,-1.550217,0.0
246,South Africa,-0.284670,-0.256947,-0.665642,-0.223698,0.086235,0.744892,1.0
247,Zambia,-0.477707,-0.658970,0.201727,-0.498100,-0.529127,-0.047946,0.0
